In [1]:
import json
import operator

In [2]:
def extract_seed(question):
    lines = question['SPARQL'][0].split('\n')
    seed = str(lines[4].split(' ')[1]).strip("'")
        
    return seed

In [3]:
problem_phone_seeds = {
    '7026023157': {'submitted_docs': 691,
                   'extracted_docs': 0},
    '4164557000': {'submitted_docs': 582,
                   'extracted_docs': 14},
    '3345579838': {'submitted_docs': 488,
                   'extracted_docs': 12},
    '6477932052': {'submitted_docs': 403,
                   'extracted_docs': 10},
    '5015748332': {'submitted_docs': 366,
                   'extracted_docs': 15},
    '6126696637': {'submitted_docs': 363,
                   'extracted_docs': 32},
    '4028713138': {'submitted_docs': 356,
                   'extracted_docs': 4},
    '2897000184': {'submitted_docs': 322,
                   'extracted_docs': 99},
    '4804209169': {'submitted_docs': 306,
                   'extracted_docs': 60},
    '2267928604': {'submitted_docs': 304,
                   'extracted_docs': 100},
    '3103027803': {'submitted_docs': 302,
                   'extracted_docs': 2},
    '6477878168': {'submitted_docs': 302,
                   'extracted_docs': 4},
    '7202565497': {'submitted_docs': 302,
                   'extracted_docs': 20},
    '3106587382': {'submitted_docs': 300,
                   'extracted_docs': 14},
    '2125675378': {'submitted_docs': 261,
                   'extracted_docs': 64},
    '8453210101': {'submitted_docs': 221,
                   'extracted_docs': 18},
    '6074815436': {'submitted_docs': 220,
                   'extracted_docs': 18},
    '6474997664': {'submitted_docs': 217,
                   'extracted_docs': 20},
    '9015053403': {'submitted_docs': 209,
                   'extracted_docs': 0},
    '7255023008': {'submitted_docs': 155,
                   'extracted_docs': 21},
    '4149346155': {'submitted_docs': 118,
                   'extracted_docs': 46},
    '6462800740': {'submitted_docs': 112,
                   'extracted_docs': 1}
    }

In [4]:
problem_email_seeds = {
    'aylinakkayam@gmail.com': {'submitted_docs': 395,
                               'extracted_docs': 15}
    }

In [5]:
lattice_extractions_file = '../../../data/lattice_extractions/data_nyu_2016-11-14.json'
lattice_extractions = []
with open(lattice_extractions_file, 'r') as f:
    for line in f:
        temp = json.loads(line)
        lattice_extractions.append(temp)

In [6]:
chosen_questions_file = 'chosen_questions.json'
with open(chosen_questions_file, 'r') as f:
    chosen_questions = eval(f.read())

## Confirm their low extraction numbers

In [ ]:
filt = 'lattice-phone'
found_phone = {}
for seed in problem_phone_seeds.keys():
    found_phone[seed] = 0
    for entry in lattice_extractions:
        match = 0
        if filt in entry['extractions'].keys():
            for val in entry['extractions'][filt]['results']:
                if val['value'] == '+1' + seed:
                    match = 1
        if match == 1:
            found_phone[seed] += 1
            
# Yup, there do not appear to be many extractions for the chosen seeds

## Can you find better seeds?

In [ ]:
questions_path = '../../../questions/'

seeds = {}

seeds['Cluster Identification'] = {}

file_path = questions_path + 'post_cluster_identification.json'
f = open(file_path, 'r')
for line in f:
    temp = json.loads(line)
    seed = extract_seed(temp)
    seeds['Cluster Identification'][temp['id']] = seed
f.close()

seeds['Cluster Facet'] = {}

file_path = questions_path + 'post_cluster_facet.json'
f = open(file_path, 'r')
for line in f:
    temp = json.loads(line)
    seed = extract_seed(temp)
    seeds['Cluster Facet'][temp['id']] = seed
f.close()

seeds['Cluster Aggregate'] = {}

file_path = questions_path + 'post_aggregate_V2.json'
f = open(file_path, 'r')
for line in f:
    temp = json.loads(line)
    seed = extract_seed(temp)
    seeds['Cluster Aggregate'][temp['id']] = seed
f.close()

In [ ]:
all_phone_seeds = {}
filt = 'lattice-phone'

for qtype in seeds.keys():
    for question_id in seeds[qtype]:
        if '@' not in seeds[qtype][question_id]:
            phone_seed = seeds[qtype][question_id]
            if phone_seed not in all_phone_seeds.keys():
                all_phone_seeds[phone_seed] = 0
                for entry in lattice_extractions:
                    match = 0
                    if filt in entry['extractions'].keys():
                        for val in entry['extractions'][filt]['results']:
                            if phone_seed in val['value']:
                                match = 1
                    if match == 1:
                        all_phone_seeds[phone_seed] += 1

In [ ]:
inverse = [(value, key) for key, value in all_phone_seeds.items()]
print max(inverse)

inverse = [(value, key) for key, value in found_phone.items()]
print max(inverse)

# Nope, '6126696637' is still the best matching phone seed, and it
# is already represented

## Spot check a few submissions

In [ ]:
submission_path = '../../team_submissions/'

## Spot Checking Uncharted Data

In [ ]:
file_path = submission_path + ('Uncharted/DomainDiscovery/uncharted_NYU_DD.json')
f = open(file_path, 'r')
all_data = eval(f.read())
uncharted_data = []
for entry in all_data:
    if entry['questionType'] == 'Cluster Identification':
        uncharted_data.append(entry)

In [ ]:
qtype = 'Cluster Identification'
id_pos = 0

team = 'Uncharted'

for question_id in chosen_questions['NYU'][qtype].keys():
    assumed_submissions = chosen_questions['NYU'][qtype][question_id]['submissions']
    for entry in uncharted_data:
        if entry['questionType'] == qtype:
            if entry['question_id'] == question_id:
                if 'answers' in entry.keys():
                    for thing in entry['answers'][:100]:
                        if thing[id_pos] in assumed_submissions:
                            print "{0} found".format(thing[id_pos])
                        else:
                            print "SUBMITTED DOC ID {0} MISSING.".format(thing[id_pos])

## Spot Checking ISI Data

In [ ]:
file_path = submission_path + ('ISI/DomainDiscovery/'
                               'isi-nyu-answers-dig-extractions/'
                               'properly_formatted_submissions/'
                               'formatted_post_cluster_identification'
                               '-parsed_fixed_all_answers.json')
f = open(file_path, 'r')
isi_data = eval(f.read())

In [ ]:
qtype = 'Cluster Identification'
id_pos = 0

team = 'ISI'

for question_id in chosen_questions['NYU'][qtype].keys():
    assumed_submissions = chosen_questions['NYU'][qtype][question_id]['submissions']
    for entry in uncharted_data:
        if entry['questionType'] == qtype:
            if entry['question_id'] == question_id:
                if 'answers' in entry.keys():
                    for thing in entry['answers'][:100]:
                        if thing[id_pos] in assumed_submissions:
                            print "{0} found".format(thing[id_pos])
                        else:
                            print "SUBMITTED DOC ID {0} MISSING.".format(thing[id_pos])

In [ ]:
isi_data[0]['question_id']

In [12]:
chosen_questions['JPL']['Cluster Aggregate'].keys()

['76',
 '71',
 '69',
 '82',
 '83',
 '100',
 '84',
 '78',
 '79',
 '44',
 '37',
 '35',
 '5',
 '4',
 '68']